In [163]:
import numpy as np
import cvxpy as cp

In [157]:
def poly_cvar_ball(p,alpha,x,f,r):   #### Function for Calculating Robust CVaR with Polynomial Divergence
    n = len(x)
    theta_1 = cp.Variable(1)
    theta_2 = cp.Variable(1)
    lbda = cp.Variable(1, nonneg = True)
    t = cp.Variable(n)
    z = cp.Variable(n, nonneg= True)
    s = cp.Variable(n)
    constraints = [lbda-s*(1-p) <= z, theta_1 + cp.pos((1 / alpha) * (theta_2 - x)) <= s]
    macht = np.array([(p-1)/p, 1-(p-1)/p])
    for i in range(n):
        constraints.append(z[i] <= cp.geo_mean(cp.vstack([lbda+p*t[i],lbda]),macht)) 
    obj = cp.Minimize(-theta_1-theta_2+lbda*r+t@f)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value, theta_1.value, theta_2.value, lbda.value)


def poly_cvar_ball_IS(p, alpha, x, f, r):   #### With importance sampling
    n = len(x)
    theta_1 = cp.Variable(1)
    theta_2 = cp.Variable(1)
    lbda = cp.Variable(1, nonneg = True)
    t = cp.Variable(n)
    z = cp.Variable(n, nonneg= True)
    s = cp.Variable(n)
    constraints = [lbda-s*(1-p) <= z, theta_1 + cp.pos((1 / alpha) * (theta_2 - x)) <= s]
    macht = np.array([(p-1)/p, 1-(p-1)/p])
    for i in range(n):
        constraints.append(z[i] <= cp.geo_mean(cp.vstack([lbda+p*t[i],lbda]),macht)) 
    obj = cp.Minimize(-theta_1-theta_2+lbda*r+t@(cp.multiply(f,2/np.abs(x))))
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value, theta_1.value, theta_2.value, lbda.value)

def kl_cvar_ball(alpha, x, f, r):  #### Function for Calculating Robust CVaR with KL-Divergence
    n = len(x)
    theta_1 = cp.Variable()
    theta_2 = cp.Variable()
    lbda = cp.Variable(nonneg=True)
    t = cp.Variable(n)
    w = cp.Variable(n)
    s = cp.Variable(n)
    constraints = []
    constraints.append(theta_1 + cp.pos(1 / alpha * (theta_2 - x)) <= s)
    constraints.append(w - lbda <= t)
    constraints.append(cp.kl_div(lbda, w) + lbda + s - w <= 0)
    obj = cp.Minimize(-theta_1 - theta_2 + lbda * r + cp.sum(cp.multiply(t, f)))
    prob = cp.Problem(obj, constraints)
    prob.solve(solver=cp.MOSEK)
    return prob.value, theta_1.value, theta_2.value, lbda.value

def kl_cvar_ball_IS(alpha, x, f, r):   #### With importance sampling
    n = len(x)
    theta_1 = cp.Variable()
    theta_2 = cp.Variable()
    lbda = cp.Variable(nonneg=True)
    t = cp.Variable(n)
    w = cp.Variable(n)
    s = cp.Variable(n)
    constraints = []
    constraints.append(theta_1 + cp.pos(1 / alpha * (theta_2 - x)) <= s)
    constraints.append(w - lbda <= t)
    constraints.append(cp.kl_div(lbda, w) + lbda + s - w <= 0)
    obj = cp.Minimize(-theta_1 - theta_2 + lbda * r + cp.sum(cp.multiply(t, cp.multiply(f,2/np.abs(x)))))
    prob = cp.Problem(obj, constraints)
    prob.solve(solver=cp.MOSEK)
    return prob.value, theta_1.value, theta_2.value, lbda.value

In [119]:
np.random.seed(1)    
N = np.arange(500,6500,500)
poly_r = np.zeros(len(N))
kl_r = np.zeros(len(N))
X = -(np.random.pareto(2,size = 6500 )+1)
for i in range(len(N)):
    n = N[i]
    poly_r[i] = poly_cvar_ball(3,0.025,X[0:n],np.zeros(n)+1/n,0.02)[0]
    kl_r[i] = kl_cvar_ball(1-0.975,X[0:n],np.zeros(n)+1/n,0.02)[0]
    print('n:', n, 'Polynomial:',poly_r[i], 'KL:',kl_r[i])

n: 500 Polynomial: 13.370730928596737 KL: 16.403631682358068
n: 1000 Polynomial: 13.443091755450231 KL: 16.18146773548914
n: 1500 Polynomial: 14.8809067125657 KL: 19.037343292934864
n: 2000 Polynomial: 15.411312677457088 KL: 19.911044502456193
n: 2500 Polynomial: 15.238795857171933 KL: 20.138910313259828
n: 3000 Polynomial: 20.17242069501168 KL: 39.276706807879826
n: 3500 Polynomial: 19.913979526348427 KL: 38.32994647875242
n: 4000 Polynomial: 19.357133891206526 KL: 37.32481244564693
n: 4500 Polynomial: 19.272346096001208 KL: 36.73109617785556
n: 5000 Polynomial: 18.765057737382417 KL: 35.93147230035033
n: 5500 Polynomial: 18.92678146160716 KL: 35.61177657136223
n: 6000 Polynomial: 18.560383695897734 KL: 35.01297555792823


In [159]:
np.random.seed(1)                        #### Same experiment, but with importance sampling
N = np.arange(500,6500,500)
poly_r_IS = np.zeros(len(N))
kl_r_IS = np.zeros(len(N))
X = -(np.random.pareto(1,size = 6500)+1)
for i in range(len(N)):
    n = N[i]
    poly_r_IS[i] = poly_cvar_ball_IS(3,0.025,X[0:n],np.zeros(n)+1/n,0.02)[0]
    kl_r_IS[i] = kl_cvar_ball_IS(1-0.975,X[0:n],np.zeros(n)+1/n,0.02)[0]
    print('n:', n, 'Polynomial:',poly_r_IS[i], 'KL:',kl_r_IS[i])

n: 500 Polynomial: 24.94327481908894 KL: 74.371292292464
n: 1000 Polynomial: 24.68987552165416 KL: 71.14868874236211
n: 1500 Polynomial: 25.34343512063141 KL: 99.90949329539924
n: 2000 Polynomial: 25.794405249255583 KL: 102.736962506664
n: 2500 Polynomial: 24.69853959849035 KL: 106.56994044505761
n: 3000 Polynomial: 25.441941292811904 KL: 713.809122621102
n: 3500 Polynomial: 25.63285902898825 KL: 703.4376958955236
n: 4000 Polynomial: 25.528549464363877 KL: 694.5761277424838
n: 4500 Polynomial: 25.589023517086865 KL: 686.2830332677058
n: 5000 Polynomial: 25.518881724907768 KL: 680.2619069612775
n: 5500 Polynomial: 25.61230928248229 KL: 674.3447190292198
n: 6000 Polynomial: 25.680454008268434 KL: 669.0033308984907
